In [ ]:
import tensorflow as tf

if tf.test.gpu_device_name():
    print(f"GPU is available: {tf.test.gpu_device_name()}")
else:
    print("GPU not available.")

## 3. Загрузка и предобработка графа

### Конвертация OWL в формат триплетов
Используем библиотеку rdflib для извлечения триплетов.

In [2]:
from rdflib import Graph

# Загрузка графа OWL
graph = Graph()
graph.parse("skyrim_knowledge_graph.owl", format="xml")

# Извлечение триплетов
triplets = []
for s, p, o in graph:
    triplets.append((str(s), str(p), str(o)))

# Сохранение триплетов в формате CSV
import pandas as pd

triplets_df = pd.DataFrame(triplets, columns=["subject", "predicate", "object"])
triplets_df.to_csv("skyrim_triplets.csv", index=False)

print(f"Extracted {len(triplets)} triplets.")

Extracted 18605 triplets.


## 4. Разбиение данных
### 4.1 Подготовка данных
Загрузим данные и удалим строки с отсутствующими значениями.

In [3]:
# Загрузка триплетов
triplets = pd.read_csv("skyrim_triplets.csv")

# Удаление null значений
triplets.dropna(inplace=True)

In [4]:
triplets

,subject,predicate,object
0,http://example.org/skyrim#Nord_Hero_Battle_Axe,http://example.org/skyrim#madeOf,Steel Ingot
1,http://example.org/skyrim#Ancarion,http://example.org/skyrim#hasSkill,http://example.org/skyrim#Sneak
2,http://example.org/skyrim#Valerica,http://example.org/skyrim#hasHealth,150+(PC-1)×9
3,http://example.org/skyrim#Bolfrida_Brandy-Mug,http://example.org/skyrim#hasRace,http://example.org/skyrim#Nord
4,http://example.org/skyrim#Brand-Shei,http://example.org/skyrim#isMemberOf,http://example.org/skyrim#JobMerchantFaction
...,...,...,...
18600,http://example.org/skyrim#Forsworn_Axe,http://example.org/skyrim#damage,11
18601,http://example.org/skyrim#Jod,http://example.org/skyrim#hasRace,http://example.org/skyrim#Nord
18602,http://example.org/skyrim#Chillrend_(Lvl_11-18),http://example.org/skyrim#requiresSkill,One-Handed
18603,http://example.org/skyrim#Nightingale_Blade_(L...,http://example.org/skyrim#weight,14.0


## Обучение модели

In [5]:
import numpy as np
from ampligraph.latent_features import ScoringBasedEmbeddingModel
from ampligraph.evaluation import train_test_split_no_unseen
from ampligraph.latent_features.loss_functions import get as get_loss
from ampligraph.latent_features.regularizers import get as get_regularizer

In [6]:
# Разделение данных для обучения и валидации
train_triplets, valid = train_test_split_no_unseen(triplets.values, test_size=0.2)

print(f"Training triplets: {len(train_triplets)}, Testing triplets: {len(valid)}")

# Инициализация модели
model = ScoringBasedEmbeddingModel(scoring_type='ComplEx',  k=500,
               eta=10, seed=32)

# Optimizer, loss and regularizer definition
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
loss = get_loss('multiclass_nll')
regularizer = get_regularizer('LP', {'p': 3, 'lambda': 1e-5})

model.compile(optimizer=optimizer, loss=loss, entity_relation_regularizer=regularizer)
# Обучение модели
model.fit(train_triplets,epochs=400,verbose=True)

Training triplets: 14844, Testing triplets: 3710
Epoch 1/400
16/16 [==============================] - 10s 639ms/step - loss: 2372.9592
Epoch 2/400
16/16 [==============================] - 7s 430ms/step - loss: 2372.8218
Epoch 3/400
16/16 [==============================] - 6s 394ms/step - loss: 2372.6975
Epoch 4/400
16/16 [==============================] - 10s 626ms/step - loss: 2372.5728
Epoch 5/400
16/16 [==============================] - 5s 316ms/step - loss: 2372.4465
Epoch 6/400
16/16 [==============================] - 6s 348ms/step - loss: 2372.3164
Epoch 7/400
16/16 [==============================] - 5s 283ms/step - loss: 2372.1799
Epoch 8/400
16/16 [==============================] - 5s 289ms/step - loss: 2372.0344
Epoch 9/400
16/16 [==============================] - 5s 325ms/step - loss: 2371.8801
Epoch 10/400
16/16 [==============================] - 5s 323ms/step - loss: 2371.7129
Epoch 11/400
16/16 [==============================] - 5s 283ms/step - loss: 2371.5317
Epoch 12/400

In [7]:
from ampligraph.evaluation import train_test_split_no_unseen

# Оценка качества
ranks = model.evaluate(valid, verbose=True, use_filter={'train': train_triplets,
                                                        'test': valid})
print("Ранговые метрики:", ranks)

372/372 [==============================] - 113s 303ms/step
Ранговые метрики: [[ 109  637]
 [   2    1]
 [   3  567]
 ...
 [ 782   21]
 [4146 2782]
 [   2    5]]


In [8]:
from ampligraph.evaluation import mr_score, mrr_score, hits_at_n_score

mr = mr_score(ranks)
mrr = mrr_score(ranks)

print("MRR: %.2f" % (mrr))
print("MR: %.2f" % (mr))

hits_10 = hits_at_n_score(ranks, n=10)
print("Hits@10: %.2f" % (hits_10))
hits_3 = hits_at_n_score(ranks, n=3)
print("Hits@3: %.2f" % (hits_3))
hits_1 = hits_at_n_score(ranks, n=1)
print("Hits@1: %.2f" % (hits_1))

MRR: 0.40
MR: 180.34
Hits@10: 0.55
Hits@3: 0.44
Hits@1: 0.32


## Classification

### Задача классификации:

Цель задачи: Предсказать локацию персонажа в Skyrim.
Признак для классификации: Предсказание локации персонажа по его эмбеддингу в графе.
Тип задачи: Многоклассовая классификация (каждый класс — это одна уникальная локация).
Сколько классов: Количество уникальных локаций (например, 10-20 локаций). Для проверки можно вывести распределение классов.

### Для чего решаем эту задачу:

Цель этой задачи — предсказать, в какой локации находится персонаж на основе его связи с другими сущностями в графе. Это может быть полезно, например, в игре, для оптимизации поиска информации о локациях или для анализа поведения персонажей в разных частях игры. Кроме того некоторые персонажи дают полезные уникальные предметы или навыки и найти их не всегда легко. Информацию о местоположении персонажа не всегда указана даже на вики.

Определение принадлежности к локации:

Признак: http://example.org/skyrim#locatedAt.
Цель: Предсказать локацию персонажа, основываясь на его связях в графе.

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [10]:
# Получение эмбеддингов
characters = [row[0] for row in train_triplets if row[1] == 'http://example.org/skyrim#locatedAt']
locations = list(set([row[2] for row in train_triplets if row[1] == 'http://example.org/skyrim#locatedAt']))

In [11]:
character_embeddings = model.get_embeddings(characters, embedding_type='e')
labels = [locations.index(loc) for loc in [row[2] for row in train_triplets if row[1] == 'http://example.org/skyrim#locatedAt']]

In [12]:
# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(character_embeddings, labels, test_size=0.2)

# Обучение классификатора
clf = LogisticRegression()
clf.fit(X_train, y_train)

# Оценка
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         1
           2       0.00      0.00      0.00         1
           5       0.33      1.00      0.50         1
           8       1.00      1.00      1.00         1
           9       0.00      0.00      0.00         2
          10       0.00      0.00      0.00         1
          11       0.00      0.00      0.00         0
          12       0.00      0.00      0.00         1
          13       0.00      0.00      0.00         1
          17       0.00      0.00      0.00         1
          19       0.00      0.00      0.00         0
          20       1.00      0.50      0.67         2
          30       0.00      0.00      0.00         1
          31       0.00      0.00      0.00         1
          33       1.00      1.00      1.00         1
          39       0.00      0.00      0.00         0
          42       0.00    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

# SPARKL Запросы

In [1]:
from rdflib import Graph

In [2]:
g = Graph()
g.parse(r"skyrim_knowledge_graph.owl")

<Graph identifier=N18ea9cdfa6f142688073b9e77c5730fd (<class 'rdflib.graph.Graph'>)>

In [135]:
def execute_query(query) -> list[str]:
    pref = "http://example.org/skyrim#"
    answers = g.query(query)
    result = []
    for answer in answers:
        result.append(" | ".join(map(lambda x: str(x).replace(pref, ''), answer)))
    return result

def execute_query_raw(query):
    answers = g.query(query)
    return answers

In [136]:
# Получаем список всех городов
cities_query = """
PREFIX base: <http://example.org/skyrim#>
SELECT DISTINCT ?city WHERE { ?city rdf:type base:City }
"""

# Получаем список всех фракций
factions_query = """
PREFIX base: <http://example.org/skyrim#>
SELECT DISTINCT ?faction WHERE { ?faction rdf:type base:Faction }
"""

# Получаем список всех навыков
skills_query = """
PREFIX base: <http://example.org/skyrim#>
SELECT DISTINCT ?skill WHERE { ?skill rdf:type base:Skill }
"""

print("Cities:", execute_query(cities_query))
print("Factions:", execute_query(factions_query))
print("Skills:", execute_query(skills_query))

Cities: ['Whiterun', 'Darkwater_Crossing', 'Narzulbur', 'Temple_of_Miraak,_Skaal_Village', 'any_city_captured_by_the_Stormcloaks', 'Karthwasten', 'Riften_(After_his_quest)', 'Markarth', 'Winterhold', 'Dushnikh_Yal', 'Dragon_Bridge', 'Riften', 'Raven_Rock', 'Raven_Rock,_Tel_Mithryn', 'Skaal_Village', 'Stonehills', 'Solitude_then_Windhelm', "Shor's_Stone", 'Falkreath', 'Riverwood', 'Largashbur', 'Corinthe', 'Kynesgrove', 'Morthal', 'Windhelm', 'Dawnstar', 'Ivarstead', 'Solitude', 'Tel_Mithryn', 'Helgen', 'Rorikstead', 'Mor_Khazgur']
Factions: ['Riverwood_Alvors_House_Faction', 'College_of_Winterhold_5(Master-Wizard)', 'Raven_Rock_Glover_Services_Faction', 'BYOHHousecarlFalkreathCrimeFaction', 'BYOHHousecarlHjaalmarchCrimeFaction', 'Crescius_&_Aphia_Shared_Faction', 'SolitudeBluePalaceFaction', 'DLC2SVOslafsHouseFaction', 'Fish_vendor', 'Favor154QuestGiverFaction', 'Riften_Honorhall_Orphanage_Faction', 'CrimeFactionCidhnaMine', "Raven_Rock_Crescius_Caerellius's_House_Faction", 'ServicesWh

In [137]:
# Найти всех персонажей, которые являются членами определенной фракции и владеют определенным навыком
query1 = """
PREFIX base: <http://example.org/skyrim#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT DISTINCT ?character ?faction ?skill
WHERE {
    ?character rdf:type base:Character ;
              base:isMemberOf ?faction ;
              base:hasSkill ?skill .
    FILTER(?faction = base:The_Dark_Brotherhood)
    FILTER(?skill = base:One-handed)

}
ORDER BY ?character
"""
results1 = execute_query(query1)
print(*results1, sep='\n')

Arnbjorn | The_Dark_Brotherhood | One-handed
Astrid | The_Dark_Brotherhood | One-handed
Babette | The_Dark_Brotherhood | One-handed
Cicero | The_Dark_Brotherhood | One-handed
Dark_Brotherhood_Initiate | The_Dark_Brotherhood | One-handed
Gabriella | The_Dark_Brotherhood | One-handed
Nazir | The_Dark_Brotherhood | One-handed
Veezara | The_Dark_Brotherhood | One-handed


In [138]:
# Найти все оружие определенного типа с его характеристиками и требуемыми навыками
query2 = """
PREFIX base: <http://example.org/skyrim#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT ?weapon ?damage ?weight ?type
WHERE {
    ?weapon rdf:type base:Weapon ;
           base:damage ?damage ;
           base:weight ?weight ;
           base:hasType base:Sword .
    OPTIONAL { ?weapon base:hasType ?type }
    FILTER(?damage > 13)

}
ORDER BY DESC(?damage)
"""
results2 = execute_query(query2)
print(*results2, sep='\n')

Miraak's_Sword_ | 16 | 3.0 | Sword
Dragonbone_Sword_ | 15 | 19.0 | Sword
Chillrend_(Lvl_46+) | 15 | 16.0 | Sword
Nightingale_Blade_(Lvl_46+) | 14 | 15.0 | Sword
Chillrend_(Lvl_36-45) | 14 | 15.0 | Sword
Dragonbane_(Lvl_46+) | 14 | 14.0 | Sword
Daedric_Sword | 14 | 16.0 | Sword


In [118]:
# Найти всех персонажей в определенном городе, которые связаны с квестами
query3 = """
PREFIX base: <http://example.org/skyrim#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT DISTINCT ?character ?location
WHERE {
    ?character rdf:type base:Character ;
              base:locatedIn ?location .
    ?location rdf:type base:City .
    FILTER(?location = base:Whiterun)
}
"""
results3 = execute_query(query3)
print(*results3[:20], sep='\n')

Aela_the_Huntress | Whiterun
Danica_Pure-Spring | Whiterun
Ysolda | Whiterun
Sinmir | Whiterun
Maurice_Jondrelle | Whiterun
Gerda | Whiterun
Irileth | Whiterun
Nazeem | Whiterun
Commander_Caius | Whiterun
Hrongar | Whiterun
Fianna | Whiterun
Alfhild_Battle-Born | Whiterun
Njada_Stonearm | Whiterun
Braith | Whiterun
Dagny | Whiterun
Nelkir | Whiterun
Sigurd | Whiterun
Jenassa | Whiterun
Ulfberth_War-Bear | Whiterun
Sabjorn | Whiterun


In [124]:
# Поиск связей между навыками и расами с бонусами
query4 = """
PREFIX base: <http://example.org/skyrim#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT ?race ?skill
WHERE {
    ?race rdf:type base:Race ;
         base:bonusTo ?skill .
    ?skill rdf:type base:Skill ;
          base:belongsTo ?category .
    OPTIONAL {
        ?race base:bonusValue ?bonus .
    }
}
GROUP BY ?race ?skill ?bonus
"""
results4 = execute_query(query4)
print(*results4, sep='\n')

Breton | Conjuration
Nord | Two-Handed


In [140]:
# Найти все книги, их авторов и связанные навыки
query5 = """
PREFIX base: <http://example.org/skyrim#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT ?book ?author ?relatedSkill ?cost
WHERE {
    ?book rdf:type base:Book ;
         base:cost ?cost .
    OPTIONAL { ?book base:authoredBy ?author }
    OPTIONAL { ?book base:relatedSkill ?relatedSkill }
    FILTER (xsd:integer(?cost) > 50)
}
ORDER BY DESC(xsd:integer(?cost))
"""
results5 = execute_query(query5)
print(*results5[:20], sep="\n")

Oghma_Infinium | Xarxes | false | 2500
Black_Book:_The_Hidden_Twilight | Carillius_Melfus | false | 2000
Black_Book:_The_Winds_of_Change | Liesl_Grey-Heart | false | 2000
Black_Book:_Waking_Dreams | Bilius_Felcrex | false | 2000
Black_Book:_Epistolary_Acumen | The_Transparent_One | false | 2000
Black_Book:_Untold_Legends | None | false | 2000
Black_Book:_Filament_and_Filigree | Jelketheris | false | 2000
Black_Book:_The_Sallow_Regent | Hawfip_the_Crafter | false | 2000
Saint_Jiub's_Opus | Jiub | false | 1250
The_Nightingales_Vol._2 | Gallus_Desidenius | false | 500
The_Nightingales_Vol._1 | Gallus_Desidenius | false | 500
The_Nirnoot_Missive | Sinderion | false | 250
Annals_of_the_Dragonguard | Brother_Annulus | false | 200
Nystrom's_Journal | Nystrom | false | 150
Gratian's_Journal | Gratian_Caerellius | false | 125
Ildari's_Journal | Ildari_Sarothril | false | 125
On_Apocrypha:_Delving_Pincers | None | false | 100
Wind_and_Sand | Afa-Saryat | false | 100
On_Apocrypha:_Gnashing_Blades

In [143]:
# Сложный запрос для анализа связей между фракциями и локациями
query6 = """
PREFIX base: <http://example.org/skyrim#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT DISTINCT ?character ?faction ?location
WHERE {
    ?character base:isMemberOf ?faction ;
              base:locatedIn ?location .
}
ORDER BY ?faction
"""
results6 = execute_query(query6)
print(*results6[:30], sep='\n')

Grelod_the_Kind | Actors_dead_bodies_won't_trigger_World_Interactions | Riften
Edla | Actors_in_this_will_never_fill_world_interaction_aliases | Skaal_Village
Frea | Actors_in_this_will_never_fill_world_interaction_aliases | Skaal_Village
Tolfdir | Actors_in_this_will_never_fill_world_interaction_aliases | Winterhold
Morwen | Actors_in_this_will_never_fill_world_interaction_aliases | Skaal_Village
Finna | Actors_in_this_will_never_fill_world_interaction_aliases | Skaal_Village
Nikulas | Actors_in_this_will_never_fill_world_interaction_aliases | Skaal_Village
Storn_Crag-Strider | Actors_in_this_will_never_fill_world_interaction_aliases | Skaal_Village
Wulf_Wild-Blood | Actors_in_this_will_never_fill_world_interaction_aliases | Skaal_Village
Roggvir | Actors_in_this_will_never_fill_world_interaction_aliases | Solitude
Deor_Woodcutter | Actors_in_this_will_never_fill_world_interaction_aliases | Skaal_Village
Baldor_Iron-Shaper | Actors_in_this_will_never_fill_world_interaction_aliases | S

In [182]:
# Анализ взаимосвязей между навыками и локациями через персонажей:
query_skill_locations = """
PREFIX base: <http://example.org/skyrim#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT ?skill ?location (COUNT(DISTINCT ?character) as ?char_count)
WHERE {
    ?character base:hasSkill ?skill ;
              base:locatedIn ?location .
}
GROUP BY ?skill ?location
ORDER BY DESC(?char_count)
"""
results_skill_locations = execute_query(query_skill_locations)
print(*results_skill_locations[:20], sep='\n')

One-handed | Riften | 54
Sneak | Riften | 39
One-handed | Whiterun | 38
Archery | Riften | 36
Two-handed | Solitude | 34
One-handed | Markarth | 34
Two-handed | Whiterun | 33
Speech | Riften | 33
One-handed | Solitude | 31
Light_Armor | Riften | 30
Two-handed | Windhelm | 26
Block | Markarth | 25
Two-handed | Riften | 24
Archery | Whiterun | 23
Speech | Solitude | 22
Speech | Whiterun | 20
Block | Riften | 20
Two-handed | Dawnstar | 18
Block | Whiterun | 18
Smithing | Riften | 16


In [188]:
query_faction_analysis = """
PREFIX base: <http://example.org/skyrim#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT ?faction
       (COUNT(DISTINCT ?member) as ?member_count)
      #  (GROUP_CONCAT(DISTINCT COALESCE(?skill, "None"); separator=", ") as ?faction_skills)
       (COUNT(DISTINCT ?location) as ?presence_locations)
WHERE {
    ?member base:isMemberOf ?faction .
    OPTIONAL {
        ?member base:hasSkill ?skill
    }
    OPTIONAL {
        ?member base:locatedIn ?location
    }
}
GROUP BY ?faction
ORDER BY DESC(?member_count)
"""

# объяснение
"""
# 1. Объявление префиксов для упрощения записи URI
PREFIX base: <http://example.org/skyrim#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

# 2. Выбираем следующие данные:
SELECT
    ?faction                    # Идентификатор фракции
    (COUNT(DISTINCT ?member) as ?member_count)  # Количество уникальных членов
    (GROUP_CONCAT(DISTINCT COALESCE(?skill, "None"); separator=", ") as ?faction_skills)  # Список всех навыков членов фракции
    (COUNT(DISTINCT ?location) as ?presence_locations)  # Количество уникальных локаций

# 3. Условия выборки:
WHERE {
    # Основное условие: находим всех членов каждой фракции
    ?member base:isMemberOf ?faction .

    # Дополнительно (если есть) получаем навыки членов фракции
    OPTIONAL {
        ?member base:hasSkill ?skill
    }

    # Дополнительно (если есть) получаем локации членов фракции
    OPTIONAL {
        ?member base:locatedIn ?location
    }
}

# 4. Группировка по фракциям
GROUP BY ?faction

# 5. Сортировка по количеству членов (по убыванию)
ORDER BY DESC(?member_count)
"""

results_faction_analysis = execute_query(query_faction_analysis)
print(*results_faction_analysis[:20], sep='\n')

CrimeFactionRift | 102 | 5
FavorExcludedFaction | 93 | 14
CrimeFactionHaafingar | 91 | 3
CrimeFactionWhiterun | 86 | 3
CrimeFactionReach | 82 | 2
JobMerchantFaction | 76 | 21
TownRiftenFaction | 73 | 1
CrimeFactionEastmarch | 71 | 3
TownSolitudeFaction | 71 | 1
PotentialMarriageFaction | 68 | 18
TownWhiterunFaction | 67 | 1
TownMarkarthFaction | 57 | 1
PotentialFollowerFaction | 55 | 19
GuardFaction | 54 | 12
TownWindhelmFaction | 49 | 2
Blood-Kin_of_the_Orcs | 47 | 10
Skill_Trainer | 44 | 16
CrimeFactionPale | 41 | 1
Thieves_Guild_No_Pickpocketing_Faction | 39 | 7
Tribal_Orcs | 36 | 4
